# Fama-French 5 factor Model Analysis Tool

### **references :**
1. https://ssupapers.tistory.com/1
2. ChatGPT

<br>

- 원본 3 factor 코드(PAPERS tistory 블로그 제공) 활용.
- RMSE 도출하는 로직 추가.
- 두 가지 추가 요인인 RMW (Profitability)와 CMA (Investment)도 분석에 포함.

---

# I. 필요 라이브러리 호출 및 함수 정의

In [14]:
import yfinance as yf
import pandas_datareader.data as web
import pandas as pd
import statsmodels.api as sm
import numpy as np

# 월말 수익률 도출 메소드
def get_monthly_returns(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start = start_date, end = end_date, interval = '1mo')

    x = stock_data['Adj Close'].resample('M').last().pct_change().dropna()

    x.name = 'Return' # = 수익률

    return x

# 파마-프렌치 5 factor 데이터 가져오는 메소드
def get_factors(start_date, end_date):
    # pandas datareader를 사용하여 F-F_Research_Data_5_Factors_2x3 데이터를 가져옴
    ff_data = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', start=start_date, end=end_date)

    # 데이터는 딕셔너리 형태이므로 첫 번째 데이터프레임을 추출
    ff_data = ff_data[0]

    # PeriodIndex를 Timestamp로 변환
    ff_data.index = ff_data.index.to_timestamp()

    # 필요한 컬럼만 선택하여 월 단위로 리샘플링
    ff_data = ff_data[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']].resample('M').last()

    # 퍼센트 단위를 소수점으로 변환
    ff_data /= 100

    return ff_data

# 수익률과 팩터 데이터를 하나의 데이터프레임으로 병합하는 메소드
def merge_data(ticker, start_date, end_date):
    monthly_returns = get_monthly_returns(ticker, start_date, end_date)

    fama_french = get_factors(start_date, end_date)

    df = pd.merge(monthly_returns, fama_french, left_index=True, right_index=True)

    df['Excess Return'] = df['Return'] - df['RF'] # Excess Return : 초과수익률 의미

    return df

# 파마-프렌치 5 factor 모델로 OLS 회귀를 진행하여 타겟변수 ETF에 대해 분석하는 메소드
def estimate_ff_model(df):
    X = df[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]

    # 독립변수 X : 5 factors
    X = sm.add_constant(X) # 모델에 상수항(alpha = y절편)을 추가

    # 종속변수 y : 분석하고자 하는 ETF의 (무위험채권 대비) 초과수익률
    y = df['Excess Return']

    # OLS(최소자승법)을 사용해 회귀모델 적합
    model = sm.OLS(y, X).fit()

    # 예측값 계산
    predictions = model.predict(X)

    # RMSE 계산
    rmse = np.sqrt(((predictions - y) ** 2).mean())

    return model.summary(), rmse

---

# II. 각 ETF 분석 수행
- User에게 직접 ETF ticker명 이름을 input으로 받도록 수정.


In [15]:
ticker = input('분석하고자 하는 ETF의 ticker명 입력(Yahoo Finance 기준) : ')

start_date = '2000-01-01'
end_date = '2024-09-01'

df = merge_data(ticker, start_date, end_date)

model_summary, rmse = estimate_ff_model(df)
print(f'\n{model_summary}\n') ; print(f'\nRMSE : {rmse}\n')

분석하고자 하는 ETF의 ticker명 입력(Yahoo Finance 기준) : SPY


[*********************100%***********************]  1 of 1 completed
<ipython-input-14-1766148628e7>:20: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_data = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', start=start_date, end=end_date)



                            OLS Regression Results                            
Dep. Variable:          Excess Return   R-squared:                       0.986
Model:                            OLS   Adj. R-squared:                  0.986
Method:                 Least Squares   F-statistic:                     4056.
Date:                Thu, 05 Sep 2024   Prob (F-statistic):          1.47e-264
Time:                        07:30:17   Log-Likelihood:                 1122.4
No. Observations:                 294   AIC:                            -2233.
Df Residuals:                     288   BIC:                            -2211.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0003      0.000     -0.874      0.

<ipython-input-14-1766148628e7>:20: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_data = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', start=start_date, end=end_date)
